In [44]:
import os
import glob
import hashlib
import requests
import subprocess

In [45]:
def get_hash(name):
    readsize = 64 * 1024
    with open(name, 'rb') as f:
        size = os.path.getsize(name)
        data = f.read(readsize)
        f.seek(-readsize, os.SEEK_END)
        data += f.read(readsize)
    return hashlib.md5(data).hexdigest()

In [40]:
hashes = []
types = ["*mp4", "*.mkv"]

for single_type in types:
    for file in (glob.glob(single_type)):
        video = {}
        video["hash"] = (get_hash(file))
        video["name"] = (file)
        hashes.append(video)


In [62]:
headers = {
    'User-Agent': 'SubDB/1.0 (Subtest/0.1; http://github.com/not_yet)',
}

url_search = 'http://sandbox.thesubdb.com/?action=search&hash={}'
url_download = 'http://sandbox.thesubdb.com/?action=download&hash={}&language=en,es'


for video in hashes:
    response = requests.get(url_search.format(video["hash"]), headers=headers)
    if ('en' or 'es') in response.text:
        download = requests.get(url_download.format(video["hash"]), headers=headers)
        with open("{}.srt".format(video["name"][:-4]), "wb") as subtitle:
            subtitle.write(download.content)


In [63]:
#subler_command = r"{}/SublerCLI -source {} -optimize -metadata -downmix stereo -dest {}"
#cwd = os.getcwd()


#for video in hashes:
  #  subprocess.call(subler_command.format(cwd,cwd + "/" + video["name"], cwd + "/" + video["name"][0:-3] + "m4v"), cwd=os.getcwd())

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ruben/Documents/code/subtitler/SublerCLI -source /Users/ruben/Documents/code/subtitler/justified.mp4 -optimize -metadata -downmix stereo -dest /Users/ruben/Documents/code/subtitler/justified.m4v'